In [1]:
# Install and upgrade NLTK
!pip install --upgrade nltk

# Import NLTK and download everything required
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('movie_reviews')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [2]:
from nltk.corpus import movie_reviews
import pandas as pd

# Load the movie review data
documents = [(movie_reviews.raw(fileid), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# Convert to DataFrame
df = pd.DataFrame(documents, columns=['review', 'sentiment'])
df.head()

,review,sentiment
0,"plot : two teen couples go to a church party ,...",neg
1,the happy bastard's quick movie review \ndamn ...,neg
2,it is movies like these that make a jaded movi...,neg
3,""" quest for camelot "" is warner bros . ' firs...",neg
4,synopsis : a mentally unstable man undergoing ...,neg


In [4]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Updated function using .split() instead of word_tokenize()
def preprocess_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    words = text.split()  # use basic split instead of NLTK tokenizer
    cleaned = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(cleaned)

# Apply it
df['cleaned_review'] = df['review'].apply(preprocess_text)

# Display cleaned text
df[['sentiment', 'cleaned_review']].head()

,sentiment,cleaned_review
0,neg,plot two teen couple go church party drink dri...
1,neg,happy bastard quick movie review damn y2k bug ...
2,neg,movie like make jaded movie viewer thankful in...
3,neg,quest camelot warner bros first featurelength ...
4,neg,synopsis mentally unstable man undergoing psyc...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Create TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # limit to top 5000 words

# Convert text data to numerical features
X = tfidf.fit_transform(df['cleaned_review']).toarray()

# Convert sentiment labels to 0 (neg) and 1 (pos)
y = df['sentiment'].apply(lambda x: 1 if x == 'pos' else 0)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Create and train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7975

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.84      0.80       199
           1       0.83      0.76      0.79       201

    accuracy                           0.80       400
   macro avg       0.80      0.80      0.80       400
weighted avg       0.80      0.80      0.80       400


Confusion Matrix:
 [[167  32]
 [ 49 152]]


In [8]:
# Try your own review!
def predict_sentiment(review_text):
    # Preprocess the input
    cleaned = preprocess_text(review_text)
    # Transform using the same TF-IDF vectorizer
    vector = tfidf.transform([cleaned]).toarray()
    # Predict
    prediction = model.predict(vector)[0]
    return "Positive 😊" if prediction == 1 else "Negative 😡"

# Example
review = "This movie was absolutely amazing, I loved every second of it!"
print("Review:", review)
print("Predicted Sentiment:", predict_sentiment(review))

Review: This movie was absolutely amazing, I loved every second of it!
Predicted Sentiment: Positive 😊
